In [ ]:
from our_graph import TherapyGraph
import json

Connection successful!
Connected to Neo4j database successfully!


In [2]:
TherapyGraph.clean_database()


All nodes and relationships deleted successfully!
Remaining nodes: 0


0

In [3]:
TherapyGraph.delete_emotions_classes()
TherapyGraph.delete_all_relationships_types()


TherapyGraph.insert_emotions_as_nodes()
TherapyGraph.insert_relationships_types()

All emotion nodes deleted successfully!
Remaining emotion nodes: 0
All relationship nodes deleted successfully!
Remaining relationship nodes: 0
Total emotions: 77
Human emotions list created successfully!


8

In [4]:
TherapyGraph.add_username(name="John Doe", username="user_1")
TherapyGraph.add_username(name="Our_Therapist", username="therapist_1")

<Record u=<Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:206' labels=frozenset({'Username'}) properties={'name': 'Our_Therapist', 'created_at': neo4j.time.DateTime(2025, 5, 27, 8, 39, 18, 1000000, tzinfo=<UTC>), 'username': 'therapist_1'}>>

In [5]:
emotions = TherapyGraph.get_all_emotions(True)


anger
joy
happiness
sadness
fear
disgust
surprise
excitement
love
hate
anxiety
depression
curiosity
confusion
frustration
irritation
guilt
shame
pride
envy
jealousy
gratitude
hope
grief
despair
boredom
rage
relief
contentment
fury
terror
panic
worry
stress
contempt
calm
peace
awe
serenity
bliss
wonder
euphoria
melancholy
amazement
loneliness
nostalgia
regret
remorse
respect
compassion
empathy
disdain
sympathy
admiration
enthusiasm
passion
affection
tenderness
warmth
hostility
optimism
resentment
bitterness
forgiveness
acceptance
pessimism
rejection
approval
disapproval
trust
distrust
confidence
doubt
certainty
uncertainty
desire
lust


In [6]:
from langchain_openai import ChatOpenAI


In [7]:
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.3)

In [8]:
emotion_names = [emotion['name'] for emotion in emotions]
print(emotion_names)

['anger', 'joy', 'happiness', 'sadness', 'fear', 'disgust', 'surprise', 'excitement', 'love', 'hate', 'anxiety', 'depression', 'curiosity', 'confusion', 'frustration', 'irritation', 'guilt', 'shame', 'pride', 'envy', 'jealousy', 'gratitude', 'hope', 'grief', 'despair', 'boredom', 'rage', 'relief', 'contentment', 'fury', 'terror', 'panic', 'worry', 'stress', 'contempt', 'calm', 'peace', 'awe', 'serenity', 'bliss', 'wonder', 'euphoria', 'melancholy', 'amazement', 'loneliness', 'nostalgia', 'regret', 'remorse', 'respect', 'compassion', 'empathy', 'disdain', 'sympathy', 'admiration', 'enthusiasm', 'passion', 'affection', 'tenderness', 'warmth', 'hostility', 'optimism', 'resentment', 'bitterness', 'forgiveness', 'acceptance', 'pessimism', 'rejection', 'approval', 'disapproval', 'trust', 'distrust', 'confidence', 'doubt', 'certainty', 'uncertainty', 'desire', 'lust']


In [9]:
relationships_labels =  TherapyGraph.get_all_relationships()

In [10]:
relationships_labels

[<Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:173' labels=frozenset({'Relationship'}) properties={'name': 'brother', 'created_at': neo4j.time.DateTime(2025, 5, 27, 8, 39, 17, 664000000, tzinfo=<UTC>)}>,
 <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:184' labels=frozenset({'Relationship'}) properties={'name': 'dad', 'created_at': neo4j.time.DateTime(2025, 5, 27, 8, 39, 17, 580000000, tzinfo=<UTC>)}>,
 <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:189' labels=frozenset({'Relationship'}) properties={'name': 'mom', 'created_at': neo4j.time.DateTime(2025, 5, 27, 8, 39, 17, 627000000, tzinfo=<UTC>)}>,
 <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:198' labels=frozenset({'Relationship'}) properties={'name': 'sister', 'created_at': neo4j.time.DateTime(2025, 5, 27, 8, 39, 17, 700000000, tzinfo=<UTC>)}>,
 <Node element_id='4:ca7b4fdd-56c0-4663-a686-ea51332c9fec:199' labels=frozenset({'Relationship'}) properties={'name': 'friend', 'created_at': neo4j.t

In [11]:
relationships = [relatioship["name"] for relatioship in relationships_labels]

In [12]:
print(relationships)

['brother', 'dad', 'mom', 'sister', 'friend', 'colleague', 'partner', 'child']


In [13]:
def connect_emotions(analysis, i):
     # Create emotion relationships
    if 'emotions' in analysis:
        for emotion_name in analysis['emotions']:
            if emotion_name in emotion_names:
                emotion_node_id = f"emotion_{emotion_name.lower().replace(' ', '_')}"
                print(f"Connecting emotion '{emotion_name}' to message {i}")
                TherapyGraph.connect_emotion(message_id=f"msg_{i}", emotion_name = emotion_name)
            else:
                print(f"Emotion '{emotion_name}' not found in the list.")
                continue

In [14]:
from pydantic import BaseModel
from typing import List, Optional

with open('test_msg.json', 'r') as file:
    agent_message = json.load(file)


for message in agent_message["conversation"]:
    print(f"Adding message {message['message_id']} from {message['role']}")
    msg = message["message"]
    print(f"Message content: {msg}")
    username = "therapist_1" if message["role"] == "agent" else "user_1"
    msgid = message["message_id"]
    TherapyGraph.add_message_node(text=msg, conversation_id="conv_1", message_id=f"msg_{msgid}", username=username)
    
    # Use LLM to analyze the message for emotions and relationships
    class EmotionRelationshipAnalysis(BaseModel):
        emotions: Optional[List[str]] = None
        relationships: Optional[List[str]] = None

    try: 
        prompt = f"""Analyze this message to identify:
        1. Emotions expressed or implied in the message
        2. People or relationships the user is talking about
        
        Return a JSON object with:
        - 'emotions' field: list of emotions from this list {emotion_names} or null if none found
        - 'relationships' field: list of people/relationship from this list {relationships} or null if none found
        
        Message: '{msg}'
        
        Expected format: {{"emotions": ["stress"], "relationships": ["dad", "child"]}}
        Use null for fields where nothing is found."""
        
        response = llm.invoke(prompt)
        
        # Parse and validate the response using Pydantic
        analysis_data = json.loads(response.content)
        analysis = EmotionRelationshipAnalysis(**analysis_data)
        
        print(f"Analysis for message {message['message_id']}: {analysis.model_dump()}")
        
        # Connect emotions
        if analysis.emotions:
            connect_emotions(analysis.model_dump(), message['message_id'])
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        analysis = EmotionRelationshipAnalysis(emotions=None, relationships=None)
    except Exception as e:
        print(f"Error analyzing message: {e}")
        analysis = EmotionRelationshipAnalysis(emotions=None, relationships=None)

    # Connect relationships

    if analysis.relationships:
        for relationship in analysis.relationships:
            print(f"Connecting relationship '{relationship}' to message {message['message_id']}")
            TherapyGraph.connect_person(name=relationship, username=username, role="person")




Adding message 1 from agent
Message content: hello, how can i help you today? 
Analysis for message 1: {'emotions': None, 'relationships': None}
Adding message 2 from user
Message content: hi, my dad beat me when i was child i think of this sometimes like now, that makes me sad. 
Analysis for message 2: {'emotions': ['sadness'], 'relationships': ['dad', 'child']}
Connecting emotion 'sadness' to message 2
Emotion 'sadness' added to message 'msg_2' successfully!
Connecting relationship 'dad' to message 2
Connecting relationship 'child' to message 2
Adding message 3 from agent
Message content: that is not very good, how does that affect you now? 
Analysis for message 3: {'emotions': ['worry'], 'relationships': None}
Connecting emotion 'worry' to message 3
Emotion 'worry' added to message 'msg_3' successfully!
Adding message 4 from user
Message content: i get scared when he gets angry.
Analysis for message 4: {'emotions': ['fear', 'anger'], 'relationships': None}
Connecting emotion 'fear' 

In [15]:
# After the conversation analysis, summarize problems from agent messages
print("\nAnalyzing problems from agent messages...")

agent_messages = [msg for msg in agent_message["conversation"] ]

if agent_messages:
    # Prepare all agent messages for problem analysis
    all_agent_text = "\n".join([f"Message {msg['message_id']}: {msg['message']}" for msg in agent_messages])
    print(f"All agent messages for problem analysis:\n{all_agent_text}")
    class ProblemAnalysis(BaseModel):
        problems: Optional[List[Optional[dict]]] = None  # Allow None values in the list
    
    try:
        problem_prompt = f"""Analyze the following therapist/agent messages to identify specific problems or issues that the user is facing. 
        
        For each problem identified, specify which message it came from.
        
        Return a JSON object with:
        - 'problems' field: list of objects with 'name' (the problem) and 'message_id' (where it was identified) or null if none found
        
        Agent Messages:
        {all_agent_text}
        
        Expected format: {{"problems": [{{"name": "trauma", "message_id": "1"}}, {{"name": "stress", "message_id": "2"}}]}}
        Use null if no problems are identified."""
        
        problem_response = llm.invoke(problem_prompt)
        problem_data = json.loads(problem_response.content)
        problem_analysis = ProblemAnalysis(**problem_data)
        
        print(f"Problem analysis: {problem_analysis.model_dump()}")
        
        # Connect problems to graph database
        if problem_analysis.problems:
            for problem in problem_analysis.problems:
                # Skip None values that might be returned by the LLM
                if problem is None:
                    continue
                    
                # Validate that problem has the required fields
                if not isinstance(problem, dict) or 'name' not in problem or 'message_id' not in problem:
                    print(f"Skipping invalid problem format: {problem}")
                    continue
                    
                problem_name = problem['name']
                message_id = problem['message_id']
                username = "user_1"  # Since these are agent messages
                
                print(f"Connecting problem '{problem_name}' from message {message_id}")
                TherapyGraph.connect_problem(problem_name=problem_name, message_id=f"msg_{message_id}", username=username)
        else:
            print("No problems identified in the agent messages.")
                
    except json.JSONDecodeError as e:
        print(f"JSON parsing error for problems: {e}")
        print(f"Raw response content: {problem_response.content}")
    except Exception as e:
        print(f"Error analyzing problems: {e}")
        print(f"Raw response content: {problem_response.content if 'problem_response' in locals() else 'No response received'}")

     


Analyzing problems from agent messages...
All agent messages for problem analysis:
Message 1: hello, how can i help you today? 
Message 2: hi, my dad beat me when i was child i think of this sometimes like now, that makes me sad. 
Message 3: that is not very good, how does that affect you now? 
Message 4: i get scared when he gets angry.
Message 5: i understand, so this is your problem
Message 7: yes this is my problem
Problem analysis: {'problems': [{'name': 'childhood abuse', 'message_id': '2'}, {'name': 'trauma', 'message_id': '2'}, {'name': 'fear of anger', 'message_id': '4'}, {'name': 'sadness', 'message_id': '2'}]}
Connecting problem 'childhood abuse' from message 2
Problem 'childhood abuse' added to message 'msg_2' and user 'user_1' successfully!
Connecting problem 'trauma' from message 2
Problem 'trauma' added to message 'msg_2' and user 'user_1' successfully!
Connecting problem 'fear of anger' from message 4
Problem 'fear of anger' added to message 'msg_4' and user 'user_1' s

In [16]:
# Connect all messages to the conversation summary
conversation_id = "conv_1"

create_conversation = TherapyGraph.create_conversation(conversation_id=conversation_id)

print(f"Connecting all messages to conversation: {conversation_id}")

# Get all messages and connect them to the conversation
for msg in agent_message["conversation"]:
    message_id = f"msg_{msg['message_id']}"
    print(f"Connecting message {message_id} to conversation {conversation_id}")
    TherapyGraph.connect_message_to_conversation(message_id=message_id, conversation_id=conversation_id)

print(f"Successfully connected all {len(agent_message['conversation'])} messages to conversation {conversation_id}")


Conversation 'conv_1' created successfully!
Connecting all messages to conversation: conv_1
Connecting message msg_1 to conversation conv_1
Connecting message msg_2 to conversation conv_1
Connecting message msg_3 to conversation conv_1
Connecting message msg_4 to conversation conv_1
Connecting message msg_5 to conversation conv_1
Connecting message msg_7 to conversation conv_1
Successfully connected all 6 messages to conversation conv_1
